In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import sweetviz as sv
import missingno as msno
import time 
import sys
import os
import csv
from bs4 import BeautifulSoup
import requests
import re

from sklearn.compose import make_column_selector, ColumnTransformer, make_column_transformer
# sklearn.compose: The sklearn.compose module is a submodule of the sklearn library for machine learning in Python. It provides functions for creating complex preprocessing and modeling pipelines.
from sklearn.preprocessing import OneHotEncoder,StandardScaler,PolynomialFeatures,RobustScaler
#sklearn.preprocessing: The sklearn.preprocessing module is a submodule of the sklearn library that provides functions for preprocessing data, such as scaling and normalizing features, imputing missing values, and encoding categorical variables.
from sklearn.linear_model import Ridge,LinearRegression,Lasso, ElasticNet
# sklearn.linear_model: The sklearn.linear_model module is a submodule of the sklearn library that provides functions for fitting linear models for regression and classification.
from sklearn.pipeline import make_pipeline
# sklearn.pipeline: The sklearn.pipeline module is a submodule of the sklearn library that provides functions for creating and working with pipelines of transformers and models.
from sklearn.model_selection import train_test_split,GridSearchCV,learning_curve, RandomizedSearchCV, cross_val_score, KFold
# sklearn.model_selection: The sklearn.model_selection module is a submodule of the sklearn library that provides functions for splitting data into training and test sets, evaluating models using cross-validation, and hyperparameter tuning.
from sklearn.dummy import DummyRegressor
# sklearn.dummy: The sklearn.dummy module is a submodule of the sklearn library that provides simple dummy models for regression and classification.


from sklearn.impute import SimpleImputer
import numpy as np

from sklearn.ensemble import RandomForestClassifier
import pickle


In [3]:
try:
    df = pd.read_csv('/home/apprenant/Documents/archive/SBAnational.csv')
except:
    df = pd.read_csv('C:/Users/emada/Downloads/loan_project (1)/SBAnational.csv')

df['ApprovalFY'] = df['ApprovalFY'].replace('A', '', regex=True).astype(int)

<ipython-input-3-f0f59de9cb82>:4: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:/Users/emada/Downloads/loan_project (1)/SBAnational.csv')


In [4]:
# create a list of all numeric columns
columns_to_transform_to_int = ["DisbursementGross","BalanceGross","ChgOffPrinGr","GrAppv","SBA_Appv"]


for col in columns_to_transform_to_int:
    df[col] = df[col].str.replace("$", "")
    df[col] = df[col].str.replace(",", "")
    df[col] = df[col].astype(float)
    

df['Term'] = df['Term'].astype(int)

df['LowDoc'] = df['LowDoc'].replace({'0': 'N'})
df = df[df['LowDoc'].isin(['N','Y'])]
df['RevLineCr'] = df['RevLineCr'].replace({'0': 'N', 'T':'Y'})
df = df[df['RevLineCr'].isin(['N','Y'])]

dictionnaire =  {"11":"Agriculture, forestry, fishing and hunting","72":"Accommodation and food services","21":"Mining, quarrying, and oil and gas extraction","22":
"Utilities","23":"Construction","31":"Manufacturing","32":"Manufacturing","33":"Manufacturing","42":"Wholesale trade","44":"Retail trade","45":"Retail trade","48":" Transportation and warehousing", "49":"Transportation and warehousing", "51":"Information","52":"Finance and insurance", "53":"Real estate and rental and leasing","54":"Professional, scientific, and technical services","55":"Management of companies and enterprises","56":"Administrative and support and waste management and remediation services","61":"Educational services","62":"Health care and social assistance","71":"Arts, entertainment, and recreation","81":"Other services (except public administration)", "92": "Public administration", "0" : "Other"}

df['NAICS'] = df['NAICS'].astype(str).str[:2]

df["NAICS"] = df["NAICS"].map(dictionnaire)

cols_to_drop = ['LoanNr_ChkDgt', 'Name', 'City', 'Zip', 'Bank', 'BankState', 'ApprovalDate', 'RetainedJob','ChgOffDate', 'DisbursementDate', 'DisbursementGross', 'BalanceGross', 'SBA_Appv', 'ChgOffPrinGr']
df_cleaned = df.copy()
df_cleaned.drop(columns=cols_to_drop, inplace=True)

df_cleaned.dropna(subset=['MIS_Status'], inplace=True)


<ipython-input-4-248274118137>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].str.replace("$", "")


In [5]:
y = df_cleaned.MIS_Status
X = df_cleaned.drop(columns=['MIS_Status'])

In [6]:
def make_pipeline_to_ML(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.8, random_state=42)
    numerical_features = make_column_selector(dtype_include=np.number)
    categorical_features = make_column_selector(dtype_exclude= np.number)
    numerical_pipeline = make_pipeline(SimpleImputer(strategy='mean'),StandardScaler(with_mean=False))
    categorical_pipeline = make_pipeline(SimpleImputer(strategy='most_frequent'),OneHotEncoder(handle_unknown='ignore'))
    preprocessor = make_column_transformer((numerical_pipeline, numerical_features),
                                    (categorical_pipeline, categorical_features)
                                    )
    return preprocessor, X_train, X_test, y_train, y_test


# Create the pipeline
preprocessor, X_train, X_test, y_train, y_test = make_pipeline_to_ML(X,y)

# RandomForest
![forest](https://thumbs.gfycat.com/UncomfortableWelllitDrever-size_restricted.gif)

In [7]:
# Add the RandomForestClassifier to the pipeline
clf = make_pipeline(preprocessor, RandomForestClassifier(random_state=42, 
                                                        max_depth = 10, 
                                                        n_estimators = 200,
                                                        min_samples_split = 4800,
                                                        min_samples_leaf = 700,
                                                        n_jobs=-1,
                                                        class_weight='balanced'))

# Fit the pipeline on the training data
clf.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler(with_mean=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x00000289AE4D4D90>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                            

In [ ]:
{
  "State": "CA",
  "NAICS": "Other",
  "ApprovalFY": 1997,
  "Term": 240,
  "NoEmp": 6,
  "NewExist": 1,
  "CreateJob": 0,
  "FranchiseCode": 1,
  "UrbanRural": 0,
  "RevLineCr": "Y",
  "LowDoc": "N",
  "GrAppv": 540000
}


{'param1': 'CA',
 'param2': 'Other',
 'param3': 1997,
 'param4': 240,
 'param5': 6,
 'param6': 1,
 'param7': 0,
 'param8': 1,
 'param9': 0,
 'param10': 'Y',
 'param11': 'N',
 'param12': 540000}

In [11]:

# Save the model to a file
with open("RFR_Model.pkl", "wb") as file:
    pickle.dump(clf, file)

In [12]:
from fastapi import FastAPI

from typing import Optional
from pydantic import BaseModel
import pickle
import pandas as pd

pickle_in = open('RFR_Model.pkl', 'rb') 
forest_model =pickle.load(pickle_in)

def get_prediction(State, NAICS, ApprovalFY, Term, NoEmp, NewExist, CreateJob, FranchiseCode, UrbanRural, RevLineCr, LowDoc, GrAppv):
    x = [[State, NAICS, ApprovalFY, Term, NoEmp, NewExist, CreateJob, FranchiseCode, UrbanRural, RevLineCr, LowDoc, GrAppv]]
    df = pd.DataFrame(x, columns=['State', 'NAICS', 'ApprovalFY', 'Term', 'NoEmp', 'NewExist', 'CreateJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr', 'LowDoc', 'GrAppv'])
    y = forest_model.predict(df)[0]
    prob = forest_model.predict_proba(df)[0].tolist()
    return {'prediction': str(y), 'probability': prob}

In [13]:
x_test = [['CA', 'Other', 1997, 240, 6, 1, 0, 1, 0, 'Y', 'N', 540000]]
df_test = pd.DataFrame(x_test, columns=['State', 'NAICS', 'ApprovalFY', 'Term', 'NoEmp', 'NewExist', 'CreateJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr', 'LowDoc', 'GrAppv'])

# Make prediction using the clf model
y_pred = forest_model.predict(df_test)

# Print the prediction
print(y_pred)


['P I F']
